In [15]:
# !pip install -r ./requirements.txt

In [6]:
# !pip uninstall virny -y

In [8]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [9]:
# !pip install aif360

In [10]:
# !pip install BlackBoxAuditing==0.1.54

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_law_school'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
DATASET_NAME = 'LawSchoolDataset'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [0.1, 0.9]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'law_school_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  9430484b-bec6-42a0-ac65-11c53446a61f


## Initialize custom objects

In [9]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White


In [10]:
data_loader.X_data.shape

(20798, 11)

In [11]:
data_loader.X_data.dtypes

decile1b    float64
decile3     float64
lsat        float64
ugpa        float64
zfygpa      float64
zgpa        float64
fulltime     object
fam_inc      object
male         object
tier         object
race         object
dtype: object

In [12]:
data_loader.y_data.value_counts()

1.0    18505
0.0     2293
Name: pass_bar, dtype: int64

In [13]:
data_loader.X_data['race'].value_counts()

White        17491
Non-White     3307
Name: race, dtype: int64

In [14]:
data_loader.X_data['male'].value_counts()

1.0    11675
0.0     9123
Name: male, dtype: int64

## Run experiment iterations

### Experiment iteration 1

In [79]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [80]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [81]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 15:52:20 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.1, 0.3]',
 'model_init_seed': 100,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 15:52:20 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 7102,   593, 18841,  5078, 14172,  8064, 13554, 13401, 17015,
            18446,  6938,  3450,  9375, 19994, 16100,  4401,   142, 15143,
             2188,  4332],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.3
2023/08/06, 16:24:58: Tuning RandomForestClassifier...
2023/08/06, 16:31:10: Tuning for RandomForestClassifier is finished [F1 score = 0.6450669297827822, Accuracy = 0.8963141025641025]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [44]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.1_20230806__195840.csv',
    'tuning_results_Law_School_alpha_0.9_20230806__195951.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [45]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-07-31 14:43:04 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4]',
 'model_init_seed': 200,
 'session_uuid': '116104d6-7d63-4564-ae57-82b19b260fa5'}




2023-07-31 14:43:04 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')


Multiple alphas:   0%|          | 0/5 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.4
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_pubcov_CA/tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [15]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.1_20230806__195840.csv',
    'tuning_results_Law_School_alpha_0.9_20230806__195951.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [16]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 17:51:03 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[0.1, 0.9]',
 'model_init_seed': 300,
 'session_uuid': '9430484b-bec6-42a0-ac65-11c53446a61f'}




2023-08-06 17:51:03 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([19898, 14980, 13028,  5574, 17599,  3210, 11359, 19768, 19647,
             5966, 12835,  9832,  4098,  6607, 11751,   861, 11502, 17873,
            10882, 17470],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([19898, 14980, 13028,  5574, 17599,  3210, 11359, 19768, 19647,
             5966, 12835,  9832,  4098,  6607, 11751,   861, 11502, 17873,
            10882, 17470],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

2023-08-06 18:41:10 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
INFO:root:Successfully tested classifiers by bootstrap
2023-08-06 18:41:28 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics
INFO:root:Successfully computed predict proba metrics



[RandomForestClassifier] Metrics matrix:


,Metric,overall,male_priv,male_priv_correct,male_priv_incorrect,male_dis,male_dis_correct,male_dis_incorrect,race_priv,race_priv_correct,...,race_dis_incorrect,male&race_priv,male&race_priv_correct,male&race_priv_incorrect,male&race_dis,male&race_dis_correct,male&race_dis_incorrect,Model_Seed,Model_Name,Model_Params
0,Mean,0.136022,0.135940,0.116270,0.338744,0.136126,0.112677,0.331613,0.111918,0.097968,...,0.380218,0.123927,0.106417,0.320580,0.270183,0.229080,0.386184,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
1,Std,0.047445,0.047584,0.043186,0.092932,0.047268,0.041663,0.093993,0.042654,0.039190,...,0.097847,0.045116,0.040875,0.092750,0.073276,0.065269,0.095872,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
2,IQR,0.067773,0.067880,0.061717,0.131431,0.067637,0.059622,0.134454,0.060958,0.056069,...,0.139478,0.064395,0.058421,0.131484,0.105245,0.093699,0.137831,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
3,Aleatoric_Uncertainty,0.410666,0.414124,0.380054,0.765410,0.406262,0.361360,0.780596,0.369203,0.342608,...,0.826331,0.389674,0.357010,0.756507,0.643528,0.577687,0.829347,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
4,Overall_Uncertainty,0.430264,0.433732,0.398106,0.801063,0.425847,0.378898,0.817254,0.387216,0.359302,...,0.862369,0.408554,0.374316,0.793080,0.671083,0.602727,0.863999,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
5,Statistical_Bias,0.163988,0.161190,0.109006,0.699243,0.167551,0.104209,0.695616,0.136930,0.094500,...,0.661836,0.149976,0.100313,0.707719,0.319431,0.198063,0.661960,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
6,Jitter,0.044792,0.042988,0.028647,0.190848,0.047089,0.030786,0.183006,0.031982,0.023168,...,0.235634,0.038007,0.025710,0.176108,0.120058,0.082921,0.224869,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
7,Per_Sample_Accuracy,0.897534,0.905292,0.979237,0.142864,0.887656,0.978709,0.128571,0.924278,0.983297,...,0.172866,0.911889,0.981575,0.129263,0.738285,0.943583,0.158889,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
8,Label_Stability,0.935904,0.937056,0.958644,0.714466,0.934437,0.957417,0.742857,0.954011,0.966691,...,0.654512,0.945131,0.963253,0.741603,0.833547,0.887165,0.682222,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."
9,TPR,0.976981,0.977401,1.000000,0.000000,0.976427,1.000000,0.000000,0.987187,1.000000,...,0.000000,0.983362,1.000000,0.000000,0.888000,1.000000,0.000000,301,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_..."


2023-08-06 18:43:21 experiment_interface.py INFO    : Experiment run was successful!
INFO:root:Experiment run was successful!


### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.1_20230806__195840.csv',
    'tuning_results_Law_School_alpha_0.9_20230806__195951.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [46]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.1_20230806__195840.csv',
    'tuning_results_Law_School_alpha_0.9_20230806__195951.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Law_School_alpha_0.1_20230806__195840.csv',
    'tuning_results_Law_School_alpha_0.9_20230806__195951.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)